## **1 кейс**

**Система автопроверки**

**Важно**

Перед началом решения выполните следующую ячейку, чтобы загрузить два файла - эталонный код и код пользователя.

In [19]:
!wget -N https://gist.github.com/Vs8th/aafed81d81678c807a3ad3dbf93588b2/raw/user.py

!wget -N https://gist.github.com/Vs8th/95f7897019a4213c76e5b65234d31e30/raw/etalon.py

!wget -N https://gist.github.com/Vs8th/59f797dfbd33f9a4be1e5ed43bb42f4d/raw/res_cor.txt

--2025-03-26 06:28:33--  https://gist.github.com/Vs8th/aafed81d81678c807a3ad3dbf93588b2/raw/user.py
Resolving gist.github.com (gist.github.com)... 140.82.121.3
Connecting to gist.github.com (gist.github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/aafed81d81678c807a3ad3dbf93588b2/raw/user.py [following]
--2025-03-26 06:28:33--  https://gist.githubusercontent.com/Vs8th/aafed81d81678c807a3ad3dbf93588b2/raw/user.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 106 [text/plain]
Saving to: ‘user.py’

user.py             100%[===================>]     106  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2025-03-26 06:28:33

Давайте сразу посмотрим с каким кодом мы имеем дело.

In [ ]:
with open('etalon.py', 'r') as f:
    lines = f.readlines()

lines

['def add(a, b):\n', '    return a + b']

In [ ]:
with open('user.py', 'r') as f:
    lines2 = f.readlines()

lines2

['def add_numbers(a, b):\n',
 '    return a - b  # Ошибка: нужно сложить, а не вычесть']

### **Решения**

#### **Задача 1**

Задача не из легких, поэтому начнем по шагам. Для начала напишите функцию `run_tests`, которая прочтет эталонный код и код пользователя из `.py` файлов, выполнит их и сохранит результаты в файл `output.txt`.

**Решение**

Напишите свое решение ниже

In [20]:
import importlib

def run_tests(standard_code, user_code, test_cases, output_file):
  with open(output_file, 'w') as of:
    for a, b in test_cases:
      correct_result = standard_code.add(a, b)
      try:
        user_result = user_code.add_numbers(a, b)
        if user_result != correct_result:
          of.write(f'Тест не пройден\nОжидаемый результат: {correct_result}\nРезультат пользователя: {user_result}\n')
        else:
          of.write(f'Тест пройден\n')
      except AttributeError as err:
        of.write(f'Ошибка при выполнении кода пользователя: {err}')
        break

# Чтение кодов из файлов
standard_module = importlib.import_module('etalon')
user_module = importlib.import_module('user')

test_cases = [(1, 2), (3, 4), (5, 6)]
output_file = 'output.txt'
run_tests(standard_module, user_module, test_cases, output_file)


✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [21]:
with open('output.txt', 'r') as file1:
    res1 = file1.read()

with open('res_cor.txt', 'r') as file2:
    res2 = file2.read()

try:
    assert res1 == res2
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 2**

Теперь давайте двигаться дальше. Нужно добавить проверку на вредоносный код. Напишите отдельную функцию - `check_for_malicious_code`, которую мы будем вызывать перед проверкой кода.

Перед написанием кода, выполните следующую ячейку, в ней скачивается новый код пользователя, который проверит работу функции `check_for_malicious_code`.

In [ ]:
!wget -N https://gist.github.com/Vs8th/9f6a2c73c755cfcc088c7835ec4fb2c0/raw/user_malicious.py

--2025-03-26 06:08:30--  https://gist.github.com/Vs8th/9f6a2c73c755cfcc088c7835ec4fb2c0/raw/user_malicious.py
Resolving gist.github.com (gist.github.com)... 140.82.121.4
Connecting to gist.github.com (gist.github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/9f6a2c73c755cfcc088c7835ec4fb2c0/raw/user_malicious.py [following]
--2025-03-26 06:08:30--  https://gist.githubusercontent.com/Vs8th/9f6a2c73c755cfcc088c7835ec4fb2c0/raw/user_malicious.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117 [text/plain]
Saving to: ‘user_malicious.py’

user_malicious.py   100%[===================>]     117  --.-KB/s    in 0s      

Last-modified header missing -- tim

**Решение**

Напишите свое решение ниже

In [93]:
import importlib, ast, inspect

FORBIDDEN_NAMES = ['eval', 'exec']

def check_for_malicious_code(code, output_file):
  with open(output_file, 'w') as of:
    tree = ast.parse(code)
    try:
      for node in ast.walk(tree):
        # Проверка вызовов запрещенных функций
        if isinstance(node, ast.Call):
          # Проверяем, вызывается ли именно eval
          if isinstance(node.func, ast.Name) and node.func.id in FORBIDDEN_NAMES:
            raise ValueError(f'Код содержит вызов {node.func.id}()')

        # Проверка запрещенных атрибутов
        if isinstance(node, ast.Attribute) and node.attr in FORBIDDEN_NAMES:
          raise ValueError(f'Код содержит запрещенный атрибут: {node.attr}')


    except ValueError as err:
      return err.args[0]

    except SyntaxError as e:
        of.write(f"Код содержит синтаксическую ошибку: {type(e).__name__}\n")
        return f"Код содержит синтаксическую ошибку: {type(e).__name__}"

def run_tests(standard_code, user_code, test_cases, output_file):
  with open(output_file, 'w') as of:
    for a, b in test_cases:
      correct_result = standard_code.add(a, b)
      try:
        user_result = user_code.add_numbers(a, b)
        if user_result != correct_result:
          of.write(f'Тест не пройден\nОжидаемый результат: {correct_result}\nРезультат пользователя: {user_result}\n')
        else:
          of.write(f'Тест пройден\n')
      except AttributeError as err:
        of.write(f'Ошибка при выполнении кода пользователя: {err}')
        break

# Чтение кодов из файлов
standard_module = importlib.import_module('etalon')
user_module = importlib.import_module('user_malicious')

test_cases = [(1, 2), (3, 4), (5, 6)]
output_file = 'output2.txt'
# Проверка на вредоносный код
malicious_code_message = check_for_malicious_code(inspect.getsource(user_module), output_file)
if malicious_code_message:
    with open(output_file, 'w') as f:
        f.write("Ошибка при проверке модуля user_malicious:\n")
        f.write(malicious_code_message)
else:
    run_tests(standard_module, user_module, test_cases, output_file)


✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [90]:
# Здесь будет скачиваться файл с эталонным ответом

!wget -N https://gist.github.com/Vs8th/5197125c5ef1e1b34b7d73adccdaf4bb/raw/cor_output_2.txt

--2025-03-26 08:19:49--  https://gist.github.com/Vs8th/5197125c5ef1e1b34b7d73adccdaf4bb/raw/cor_output_2.txt
Resolving gist.github.com (gist.github.com)... 140.82.121.3
Connecting to gist.github.com (gist.github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/5197125c5ef1e1b34b7d73adccdaf4bb/raw/cor_output_2.txt [following]
--2025-03-26 08:19:49--  https://gist.githubusercontent.com/Vs8th/5197125c5ef1e1b34b7d73adccdaf4bb/raw/cor_output_2.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 107 [text/plain]
Saving to: ‘cor_output_2.txt’

cor_output_2.txt    100%[===================>]     107  --.-KB/s    in 0s      

Last-modified header missing -- time-st

In [91]:
with open('output2.txt', 'r') as file1:
    res1 = file1.read()

with open('cor_output_2.txt', 'r') as file2:
    res2 = file2.read()

try:
    assert res1== res2
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 3**

Движемся дальше - последнее, что нам нужно проверить - время выполнения. Если время выполнения превышает 5 секунд, уведомите об этом пользователя и остановите выполнение проверки.

Перед написанием кода, выполните следующую ячейку, в ней скачивается новый код пользователя, который проверит работу проверки на время выполнения.

In [ ]:
!wget https://gist.github.com/Vs8th/4aba49489a8b0843ef85c4a1a93f923d/raw/user_inf.py

**Решение**

Напишите свое решение ниже

In [ ]:

FORBIDDEN_NAMES = ['eval']

def check_for_malicious_code(code, output_file):

    ...

def run_tests(standard_code, user_code, test_cases, output_file):

    # сюда можете вставить ваш код из прошлой задачи
    # в него необходимо будет добавить проверку на время выполнения
    ...


# Чтение кодов из файлов
standard_module = ...
user_module = ...

test_cases = [(1, 2), (3, 4), (5, 6)]
output_file = 'output3.txt'
# Проверка на вредоносный код
malicious_code_message = check_for_malicious_code(inspect.getsource(user_module), output_file)
if malicious_code_message:
    with open(output_file, 'w') as f:
        f.write("Ошибка при проверке модуля user_malicious:\n")
        f.write(malicious_code_message)
else:
    run_tests(...)


✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/cfcafbd2cd46e8905555b02eae78406e/raw/cor_res_3.txt

In [ ]:
with open('output3.txt', 'r') as file1:
    res1 = file1.read()

with open('cor_res_3.txt', 'r') as file2:
    res2 = file2.read()

try:
    assert res1 == res2
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!
